# 03 - Simple RAG

This notebook implements the complete Simple RAG architecture:

```
User Query → Retriever → LLM + Prompt → Response
```

We'll compare two retrieval strategies:
1. **Similarity Search** - Returns most similar documents
2. **MMR** - Balances relevance with diversity

**Prerequisites:**
- 01_setup_and_basics.ipynb
- 02_embeddings_comparison.ipynb
- Vector stores created

**Duration:** ~15 minutes

**Outputs:**
- Working RAG chains
- Baseline performance metrics

## 1. Setup and Load Vector Stores

Load the pre-built vector stores from the previous notebook.

In [1]:
import sys
sys.path.append('../..')

from langchain_openai import OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_K
from shared.utils import load_vector_store, print_section_header

print_section_header("Loading Vector Store")

# Initialize embeddings
openai_embeddings = OpenAIEmbeddings()

# Load pre-built vector store
vectorstore = load_vector_store(
    OPENAI_VECTOR_STORE_PATH,
    openai_embeddings,
    verbose=True
)

print(f"\n✅ Vector store loaded and ready!")
print(f"   Using k={DEFAULT_K} documents per query")

/Users/gianlucamazza/Workspace/notebooks/llm_rag/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
USER_AGENT environment variable not set, consider setting it to identify your requests.



LOADING VECTOR STORE

✓ Loaded vector store from /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/fundamentals/../../data/vector_stores/openai_embeddings

✅ Vector store loaded and ready!
   Using k=4 documents per query


## 2. Retrieval Strategies

### Similarity Search
- Returns documents most similar to query
- Fast and straightforward
- May return redundant results

### MMR (Maximal Marginal Relevance)
- Balances relevance with diversity
- Reduces redundancy
- Better for exploration and broad topics
- Controlled by `lambda_mult`:
  - 1.0 = pure relevance
  - 0.0 = pure diversity  
  - 0.5 = balanced (recommended)

In [2]:
from shared.config import DEFAULT_MMR_FETCH_K, DEFAULT_MMR_LAMBDA

print_section_header("Creating Retrievers")

# Similarity retriever
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": DEFAULT_K}
)
print(f"✓ Similarity retriever created (k={DEFAULT_K})")

# MMR retriever
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": DEFAULT_K,
        "fetch_k": DEFAULT_MMR_FETCH_K,
        "lambda_mult": DEFAULT_MMR_LAMBDA
    }
)
print(f"✓ MMR retriever created (k={DEFAULT_K}, fetch_k={DEFAULT_MMR_FETCH_K}, λ={DEFAULT_MMR_LAMBDA})")

print("\n💡 MMR fetches {DEFAULT_MMR_FETCH_K} candidates, then selects {DEFAULT_K} diverse documents")


CREATING RETRIEVERS

✓ Similarity retriever created (k=4)
✓ MMR retriever created (k=4, fetch_k=20, λ=0.5)

💡 MMR fetches {DEFAULT_MMR_FETCH_K} candidates, then selects {DEFAULT_K} diverse documents


## 3. Test Retrieval

Compare both retrieval strategies with the same query.

In [3]:
from shared.utils import print_results

query = "What are the steps to build a RAG agent?"

print_section_header(f"Retrieval Comparison")
print(f"Query: '{query}'\n")

# Similarity search
print("=== Similarity Search ===")
similarity_docs = similarity_retriever.invoke(query)
print_results(similarity_docs, "Retrieved Documents", max_docs=3, preview_length=150)

# MMR search
print("\n" + "=" * 80)
print("\n=== MMR Search ===")
mmr_docs = mmr_retriever.invoke(query)
print_results(mmr_docs, "Retrieved Documents", max_docs=3, preview_length=150)

print("\n💡 Notice: MMR may show more diverse sources and content")


RETRIEVAL COMPARISON

Query: 'What are the steps to build a RAG agent?'

=== Similarity Search ===

Retrieved Documents
--------------------------------------------------------------------------------

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Type: web_documentation
   Date: 2025-11-12
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Rea...

2. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Type: web_documentation
   Date: 2025-11-12
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Rea...

3. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Type: web_documentation
   Date: 2025-11-12
   Content: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith 

## 4. Initialize LLM

We use GPT-4o-mini for cost-effectiveness. Set temperature=0 for deterministic responses.

In [4]:
from langchain_openai import ChatOpenAI
from shared.config import DEFAULT_MODEL, DEFAULT_TEMPERATURE

print_section_header("Initializing LLM")

llm = ChatOpenAI(
    model=DEFAULT_MODEL,
    temperature=DEFAULT_TEMPERATURE
)

print(f"✓ LLM initialized")
print(f"  Model: {DEFAULT_MODEL}")
print(f"  Temperature: {DEFAULT_TEMPERATURE} (deterministic)")


INITIALIZING LLM

✓ LLM initialized
  Model: gpt-5
  Temperature: 0.0 (deterministic)


## 5. Build RAG Chains

Create complete RAG chains using LCEL (LangChain Expression Language):

```python
Chain = Retriever → Format → Prompt + LLM → Parse
```

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from shared.prompts import RAG_PROMPT_TEMPLATE
from shared.utils import format_docs

print_section_header("Building RAG Chains")

# Similarity RAG chain
similarity_chain = (
    {"context": similarity_retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)
print("✓ Similarity RAG chain created")

# MMR RAG chain
mmr_chain = (
    {"context": mmr_retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)
print("✓ MMR RAG chain created")

print("\n✅ RAG chains ready for queries!")


BUILDING RAG CHAINS

✓ Similarity RAG chain created
✓ MMR RAG chain created

✅ RAG chains ready for queries!


## 6. Test RAG Chains

Compare answers from both retrieval strategies.

In [6]:
import time

query = "How to build a RAG agent with LangChain?"

print_section_header("RAG Chain Comparison")
print(f"Query: '{query}'\n")

# Similarity chain
print("=" * 80)
print("SIMILARITY RAG")
print("=" * 80)
start = time.time()
response_sim = similarity_chain.invoke(query)
time_sim = time.time() - start
print(response_sim)
print(f"\n⏱️  Time: {time_sim:.2f}s")

# MMR chain
print("\n" + "=" * 80)
print("MMR RAG")
print("=" * 80)
start = time.time()
response_mmr = mmr_chain.invoke(query)
time_mmr = time.time() - start
print(response_mmr)
print(f"\n⏱️  Time: {time_mmr:.2f}s")

print("\n" + "=" * 80)
print("\n📊 Metrics:")
print(f"   Similarity: {len(response_sim)} chars, {time_sim:.2f}s")
print(f"   MMR: {len(response_mmr)} chars, {time_mmr:.2f}s")


RAG CHAIN COMPARISON

Query: 'How to build a RAG agent with LangChain?'

SIMILARITY RAG
The provided context only gives a high-level outline (not full code or step-by-step instructions). Here’s the summary of how to build a RAG agent with LangChain based on the page:

- Set up your project
  - The page indicates sections for “Setup” and “Installation,” but does not include the concrete commands or code in the excerpt. [Used: “Setup”, “Installation” headings in the context]

- Index your data (Components → 1. Indexing)
  - Load documents
  - Split documents
  - Store documents
  [Used: “Components — 1. Indexing: Loading documents, Splitting documents, Storing documents”]

- Do retrieval and generation (Components → 2. Retrieval and Generation)
  - RAG agents: an agentic RAG approach lets the LLM decide when to call tools to retrieve information. [Used: “In the above agentic RAG formulation we allow the LLM to use its discretion in generating a tool call to help answer user queries. Thi

## 7. Multiple Query Testing

In [7]:
test_queries = [
    "What is a retriever?",
    "How do embeddings work?",
    "What are the key components of RAG?"
]

print_section_header("Multiple Query Testing")

for i, q in enumerate(test_queries, 1):
    print(f"\nQuery {i}: '{q}'")
    print("-" * 80)
    
    response = similarity_chain.invoke(q)
    print(response[:300] + "..." if len(response) > 300 else response)

print("\n✅ All queries processed successfully!")


MULTIPLE QUERY TESTING


Query 1: 'What is a retriever?'
--------------------------------------------------------------------------------
A retriever is the component that, given a user query, searches your storage (e.g., a vector store) and returns the most relevant document splits (chunks) to provide context for the model’s answer.

Citations from the context:
- “Retrieve: Given a user input, relevant splits are retrieved from stora...

Query 2: 'How do embeddings work?'
--------------------------------------------------------------------------------
The provided context only gives a high-level view of how embeddings are used in a system; it doesn’t go into the math or internals. Based on it:

- What embeddings are: They are vector (numeric) representations produced by an embeddings model (e.g., OpenAIEmbeddings with model "text-embedding-3-larg...

Query 3: 'What are the key components of RAG?'
--------------------------------------------------------------------------------
The key

## 8. Best Practices Summary

### Retrieval Strategy Selection

**Use Similarity Search when:**
- Precise factual answers needed
- Query is specific and well-defined
- Speed is priority

**Use MMR when:**
- Exploring a topic broadly
- Avoiding redundant information
- Diverse perspectives valuable

### Parameter Tuning

```python
# Similarity
k=4  # Typically 3-5 documents

# MMR
k=4  # Final count
fetch_k=20  # Candidates (3-5x of k)
lambda_mult=0.5  # Balance (0.0-1.0)
```

### Prompt Engineering

Good prompts:
- Instruct model to cite sources
- Tell model when to say "I don't know"
- Be specific about format
- Include context clearly

### Performance Optimization

- Cache vector stores (done!)
- Use batch operations for multiple docs
- Consider async for concurrent queries
- Monitor token usage and costs

## Summary

In this notebook, we:

✅ Loaded pre-built vector stores  
✅ Created similarity and MMR retrievers  
✅ Built complete RAG chains with LCEL  
✅ Compared retrieval strategies  
✅ Established baseline performance  

### Key Takeaways

- **Simple RAG** is the foundation for all advanced architectures
- **Similarity search** is fast and precise
- **MMR** provides diversity at slight cost of speed
- **LCEL** enables composable, modular chains
- **Vector store caching** saves time and money

### Performance Baseline

These metrics serve as baseline for comparing advanced architectures:
- **Latency:** ~1-2s per query
- **Quality:** Good answers for in-domain queries
- **Limitations:** 
  - No conversation memory
  - Fixed retrieval strategy
  - No self-correction
  - No web fallback

### Next Steps

Explore **Advanced Architectures** to address limitations:

- **[04_rag_with_memory.ipynb](../advanced_architectures/04_rag_with_memory.ipynb)** - Conversational RAG
- **[05_branched_rag.ipynb](../advanced_architectures/05_branched_rag.ipynb)** - Multi-query
- **[06_hyde.ipynb](../advanced_architectures/06_hyde.ipynb)** - Hypothetical docs
- **[07_adaptive_rag.ipynb](../advanced_architectures/07_adaptive_rag.ipynb)** - Intelligent routing
- **[08_corrective_rag.ipynb](../advanced_architectures/08_corrective_rag.ipynb)** - Quality checking
- **[09_self_rag.ipynb](../advanced_architectures/09_self_rag.ipynb)** - Self-reflective
- **[10_agentic_rag.ipynb](../advanced_architectures/10_agentic_rag.ipynb)** - Autonomous agents

---

**🎉 Fundamentals Complete!** You now have a solid foundation in RAG systems.